In [33]:
import openpyxl
from openpyxl import load_workbook
import numpy as np
import pandas as pd

In [9]:
pathExcel = r"G:\Drives compartilhados\Pontte Crédito\0_HOME EQUITY\0_Analises\CAROLINE BARUSSO PAGNAN - 572038126\KIT QI\calculo_fluxo.xlsx"

In [28]:
wb = load_workbook(filename=pathExcel, 
                   read_only=True)
ws = wb['PRICE_CORR']

# Read the cell values into a list of lists
data_rows = []
for row in ws['A18':'A339']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)


In [44]:

# Transform into dataframe
df = pd.DataFrame(data_rows)
df.replace('',np.nan, inplace=True)
df = df.dropna()
lastRow = len(df) + 17

In [45]:
date_rows = []
for row in ws['B18':f'B{lastRow}']:
    date_cols = []
    for cell in row:
        date_cols.append(cell.value)
    date_rows.append(date_cols)


In [48]:
df["data"] = pd.DataFrame(date_rows)
df

,0,data
0,2022-11-11,2022-11-11
1,2022-11-12,2022-11-12
2,2022-12-12,2022-12-12
3,2023-01-12,2023-01-12
4,2023-02-12,2023-02-12
...,...,...
237,2042-07-12,2042-07-12
238,2042-08-12,2042-08-12
239,2042-09-12,2042-09-12
240,2042-10-12,2042-10-12
